In [1]:
import pandas as pd

students_data = pd.read_csv("Resources/schools_complete.csv")
schools_data = pd.read_csv("Resources/students_complete.csv")

complete_data = pd.merge(students_data, schools_data, how="left", on=["school_name", "school_name"])
complete_data.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [2]:
unique_schools = len(complete_data["school_name"].unique())
unique_schools

15

In [3]:
total_students = complete_data["Student ID"].count()
total_students

39170

In [4]:
total_budget = complete_data["budget"].sum()
total_budget

82932329558

In [5]:
avg_math_score = complete_data["math_score"].mean()
avg_math_score

78.98537145774827

In [6]:
avg_reading_score = complete_data["reading_score"].mean()
avg_reading_score

81.87784018381414

In [7]:
passing_math = (complete_data[complete_data["math_score"] >= 70]["Student ID"].count() / total_students) * 100
passing_math

74.9808526933878

In [8]:
passing_reading = (complete_data[complete_data["reading_score"] >= 70]["Student ID"].count() / total_students) * 100
passing_reading

85.80546336482001

In [9]:
overall_passing = (complete_data[(complete_data["math_score"] >= 70) & (complete_data["reading_score"] >= 70)]["Student ID"].count() / total_students) * 100
overall_passing

65.17232575950983

In [10]:
district_summary = pd.DataFrame({
    "Total Schools": [unique_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_reading_score],
    "% Passing Math": [passing_math],
    "% Passing Reading": [passing_reading],
    "% Overall Passing": [overall_passing]
})

district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$82,932,329,558.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [11]:
school_type = complete_data.set_index('school_name')['type']
school_type

school_name
Huang High School     District
Huang High School     District
Huang High School     District
Huang High School     District
Huang High School     District
                        ...   
Thomas High School     Charter
Thomas High School     Charter
Thomas High School     Charter
Thomas High School     Charter
Thomas High School     Charter
Name: type, Length: 39170, dtype: object

In [12]:
per_school_count = schools_data['student_name'].count()
per_school_count

39170

In [13]:
per_school_budget = complete_data.groupby('school_name')['budget'].first()
per_school_budget

school_name
Bailey High School       3124928
Cabrera High School      1081356
Figueroa High School     1884411
Ford High School         1763916
Griffin High School       917500
Hernandez High School    3022020
Holden High School        248087
Huang High School        1910635
Johnson High School      3094650
Pena High School          585858
Rodriguez High School    2547363
Shelton High School      1056600
Thomas High School       1043130
Wilson High School       1319574
Wright High School       1049400
Name: budget, dtype: int64

In [15]:
per_std_bg = per_school_budget / total_students
per_std_bg

school_name
Bailey High School       79.778606
Cabrera High School      27.606740
Figueroa High School     48.108527
Ford High School         45.032321
Griffin High School      23.423538
Hernandez High School    77.151391
Holden High School        6.333597
Huang High School        48.778019
Johnson High School      79.005617
Pena High School         14.956804
Rodriguez High School    65.033521
Shelton High School      26.974726
Thomas High School       26.630840
Wilson High School       33.688384
Wright High School       26.790911
Name: budget, dtype: float64

In [16]:
avg_math_score = schools_data['math_score'].mean()
avg_reading_score = schools_data['reading_score'].mean()


In [17]:
passing_math_school = complete_data[complete_data["math_score"] >= 70].groupby("school_name").count()[["student_name"]]
passing_reading_school = complete_data[complete_data["reading_score"] >= 70].groupby("school_name").count()[["student_name"]]
passing_both_school = complete_data[(complete_data["math_score"] >= 70) & (complete_data["reading_score"] >= 70)].groupby("school_name").count()[["student_name"]]

In [18]:
school_summary = complete_data.set_index("school_name").sort_values("school_name")[["type", "size", "budget"]]
school_summary["Per Student Budget"] = school_summary["budget"] / school_summary["size"]


In [19]:
school_summary = school_summary.merge(passing_math_school, left_index=True, right_index=True)
school_summary = school_summary.merge(passing_reading_school, left_index=True, right_index=True, suffixes=("_math", "_reading"))
school_summary = school_summary.merge(passing_both_school, left_index=True, right_index=True)

school_summary["% Passing Math"] = (school_summary["student_name_math"] / school_summary["size"]) * 100
school_summary["% Passing Reading"] = (school_summary["student_name_reading"] / school_summary["size"]) * 100
school_summary["% Overall Passing"] = (school_summary["student_name"] / school_summary["size"]) * 100


In [36]:
school_summary = school_summary.rename(columns={
    "type": "School Type",
    "size": "Total Students",
    "budget": "Total School Budget",
    "math_score": "Average Math Score",
    "reading_score": "Average Reading Score",
    "student_name_math": "Students Passing Math",
    "student_name_reading": "Students Passing Reading",
    "student_name": "Students Passing Both"
})
city_schools_summary_df = pd.DataFrame(school_summary)

per_school_summary_df = city_schools_summary_df.copy()
per_school_summary_df.index.name = 'School Name'
per_school_summary_df['Total School Budget'] = per_school_summary_df['Total School Budget'].map('${:,}'.format)
per_school_summary_df['Per Student Budget'] = per_school_summary_df['Per Student Budget'].map('${:,}'.format)

per_school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,
Bailey High School,District,4976,"$3,124,928",$628.0,66.680064,81.933280,54.642283
Bailey High School,District,4976,"$3,124,928",$628.0,66.680064,81.933280,54.642283
Bailey High School,District,4976,"$3,124,928",$628.0,66.680064,81.933280,54.642283
Bailey High School,District,4976,"$3,124,928",$628.0,66.680064,81.933280,54.642283
Bailey High School,District,4976,"$3,124,928",$628.0,66.680064,81.933280,54.642283
...,...,...,...,...,...,...,...
Wright High School,Charter,1800,"$1,049,400",$583.0,93.333333,96.611111,90.333333
Wright High School,Charter,1800,"$1,049,400",$583.0,93.333333,96.611111,90.333333
Wright High School,Charter,1800,"$1,049,400",$583.0,93.333333,96.611111,90.333333


In [21]:
school_summary = school_summary.drop(columns=["Students Passing Math", "Students Passing Reading", "Students Passing Both"])
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,
Bailey High School,District,4976,3124928,628.0,66.680064,81.933280,54.642283
Bailey High School,District,4976,3124928,628.0,66.680064,81.933280,54.642283
Bailey High School,District,4976,3124928,628.0,66.680064,81.933280,54.642283
Bailey High School,District,4976,3124928,628.0,66.680064,81.933280,54.642283
Bailey High School,District,4976,3124928,628.0,66.680064,81.933280,54.642283
...,...,...,...,...,...,...,...
Wright High School,Charter,1800,1049400,583.0,93.333333,96.611111,90.333333
Wright High School,Charter,1800,1049400,583.0,93.333333,96.611111,90.333333
Wright High School,Charter,1800,1049400,583.0,93.333333,96.611111,90.333333


In [22]:
top_schools = school_summary.sort_values("% Overall Passing", ascending=False).head(5)
bottom_schools = school_summary.sort_values("% Overall Passing", ascending=True).head(5)


In [25]:
math_scores_by_grade = pd.pivot_table(complete_data, values="math_score", index="school_name", columns="grade", aggfunc="mean")
reading_scores_by_grade = pd.pivot_table(complete_data, values="reading_score", index="school_name", columns="grade", aggfunc="mean")


In [28]:
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]


In [37]:
school_spending_df = school_summary.copy()
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], spending_bins, labels=labels)

spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()

overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [38]:
school_spending_df

,School Type,Total Students,Total School Budget,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
school_name,,,,,,,,
Bailey High School,District,4976,3124928,628.0,66.680064,81.933280,54.642283,$585-630
Bailey High School,District,4976,3124928,628.0,66.680064,81.933280,54.642283,$585-630
Bailey High School,District,4976,3124928,628.0,66.680064,81.933280,54.642283,$585-630
Bailey High School,District,4976,3124928,628.0,66.680064,81.933280,54.642283,$585-630
Bailey High School,District,4976,3124928,628.0,66.680064,81.933280,54.642283,$585-630
...,...,...,...,...,...,...,...,...
Wright High School,Charter,1800,1049400,583.0,93.333333,96.611111,90.333333,<$585
Wright High School,Charter,1800,1049400,583.0,93.333333,96.611111,90.333333,<$585
Wright High School,Charter,1800,1049400,583.0,93.333333,96.611111,90.333333,<$585
